In [1]:
#!/usr/bin/env python
#
# Moritz Blumer | 2021-12-07 (2023-03-15)
#
# Conduct sliding window PCA with scikit-allel 


#### GET RID OF ORDER BY GT FILE, OR NOT? --> TEST IF CURRENT VERSION CAUSES PROBLEMS

#### FIX PCT INCLUDED SITES SCALE + ABRUNDEN PC % EXP VALUES

#### add check whether input samples are unique (if not len(set(input_salmpes)) == ...)

#### add [INFO]



# The minimum number of variants per window is set to 50 and can be changed below (NA will be returned for windows with fewer variants)
min_var_per_window = 50
var_threshold = 9
mean_threshold = 3
float_precision = 7

# import packages
import allel
import sys, os
import numpy as np
import pandas as pd




In [2]:
# set arguments
variant_file_path = '../test_dataset/input/genotype_file.tsv.gz'
#variant_file_path = '../test_dataset/input/sample.vcf'
metadata_path = '../test_dataset/input/metadata.tsv'

chrom = 'chr1'
pc=1
skip_monomorphic = True
output_prefix = 'test_out'
output_prefix = output_prefix.lower()
color_taxon = 'inversion_state'
w_size = 1000000
w_step = 10000
region = '1-34999238'
start = int(region.split('-')[0])
stop = int(region.split('-')[1])

In [3]:
## private functions

# main PCA function, using scikit-allel
def pca(win, w_start, w_size):
    '''
    Conduct PCA, but if (n_variants < min_var_per_window) generate empty/dummy output instead
    '''

    # trim off pos info
    win = [x[1:] for x in win]

    # get window mid for X value
    w_mid = int(w_start + w_size/2-1)

    # count variants
    n_variants = len(win)

    # if # variants passes specified threshold  
    if n_variants > min_var_per_window: # CHANGE TO: "if n_variants >= min_var_per_window:" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        w_gt_arr = np.array(win, dtype=np.int8)
        pca = allel.pca(w_gt_arr, n_components=2, copy=True, scaler='patterson', ploidy=2)
        out = [pca[0][: , 0], pca[0][: , 1], pca[1].explained_variance_ratio_[0]*100, pca[1].explained_variance_ratio_[1]*100, n_variants]

    # else create empty output
    else:
        print('[INFO] Skipped window ' + str(w_start) + '-' + str(w_start + w_size-1) + ' with ' + str(n_variants) + ' variants (threshold is ' + str(min_var_per_window) + ' variants per window)', file=sys.stderr, flush=True)
        empty_array = [None] * len(win[0])
        out = [empty_array, empty_array, None, None, n_variants]

    # append output
    w_mid_lst.append(w_mid)
    w_pca_lst.append(out[0]) if pc==1 else out[1] # depending on whether PC1 or PC2 was specified, append out[0] or out[1], respectively
    w_stats_lst.append([out[2], out[3], out[4]])


# frame for window-by-window analysis
def windowed_pca(variant_file_path, chrom, start, stop, metadata_df, pca, w_size, w_step, skip_monomorphic=True):

    global w_mid_lst, w_pca_lst, w_stats_lst

    # # initialize results containers
    w_mid_lst = []
    w_pca_lst = []
    w_stats_lst = []

    # # conduct windowed PCA using window_parser() function
    if variant_file_path.endswith('.vcf') or variant_file_path.endswith('.vcf.gz'):
        from window_parser import win_vcf
        win_vcf(variant_file_path, chrom, start, stop, metadata_df['id'], w_size, w_step, pca, skip_monomorphic=skip_monomorphic)

    elif variant_file_path.endswith('.tsv') or variant_file_path.endswith('.tsv.gz'):
        from window_parser import win_gt_file
        win_gt_file(variant_file_path, chrom, start, stop, metadata_df['id'], w_size, w_step, pca, skip_monomorphic=skip_monomorphic)

    # compile output dataframe for windowed PCA
    w_pca_df = pd.DataFrame(np.transpose(w_pca_lst), index=list(metadata_df['id']), columns=w_mid_lst)
    w_pca_df.index.names = ['id']

    # compile output dataframe for supplementary info (% variance explained, # sites per window)
    w_stats_df = pd.DataFrame(w_stats_lst, index=w_mid_lst, columns=['pct_explained_pc_1', 'pct_explained_pc_2', 'num_variants'])
    w_stats_df.index.names = ['window_mid']
    w_stats_df['pct_included_sites'] = w_stats_df['num_variants']/w_size/100

    return w_pca_df, w_stats_df



In [4]:
def main():

    #parse_arguments()

    # compile paths for the six output files
    w_pca_tsv_path =        output_prefix + '.w_pc_' + str(pc) + '.tsv.gz'
    w_stats_tsv_path =      output_prefix + '.w_stats'          + '.tsv.gz'
    w_pca_fig_html_path =   output_prefix + '.w_pc_' + str(pc) + '.html'
    w_pca_fig_pdf_path =    output_prefix + '.w_pc_' + str(pc) + '.pdf'
    w_stats_fig_html_path = output_prefix + '.w_stats'         + '.html'
    w_stats_fig_pdf_path =  output_prefix + '.w_stats'         + '.pdf'

    # set float format for text output
    pd.set_option('display.float_format', lambda x: '%.3f' % x)

    # read metadata
    from utils import read_metadata
    metadata_df = read_metadata(variant_file_path, metadata_path, taxon='species', group='species_1,species_2')

    # if there is output from a previous run read it in
    if os.path.exists(w_pca_tsv_path) and os.path.exists(w_stats_tsv_path):

        w_pca_df = pd.read_csv(w_pca_tsv_path,  sep='\t', index_col=None, na_values='NA')
        w_stats_df = pd.read_csv(w_stats_tsv_path,  sep='\t', index_col=None, na_values='NA')

    else:
        # run windowed PCA
        w_pca_df, w_stats_df = windowed_pca(variant_file_path, chrom, start, stop, metadata_df, pca, w_size, w_step, skip_monomorphic)

        # polarize windowed PCA output
        from utils import polarize
        w_pca_df = polarize(w_pca_df, var_threshold=9, mean_threshold=3, guide_samples=False) ##### ADJUST THIS

        # save output data before annotation if not already present
        w_pca_df.to_csv(w_pca_tsv_path, sep='\t', na_rep='NA', float_format='%.' + str(float_precision) + 'f', compression='gzip')
        w_stats_df.to_csv(w_stats_tsv_path, sep='\t', na_rep='NA', float_format='%.' + str(float_precision) + 'f', compression='gzip')

    # pivot windowed pca output and annotate with metadata
    from utils import annotate
    w_pca_anno_df = annotate(w_pca_df, metadata_df, 'pc_' + str(pc))

    # free up memory
    del w_pca_df

    # plot windowed PCA output & save
    from utils import plot_w_pca
    w_pca_fig = plot_w_pca(w_pca_anno_df, 1, color_taxon, chrom, start, stop, w_size, w_step)
    w_pca_fig.write_html(w_pca_fig_html_path)
    w_pca_fig.write_image(w_pca_fig_pdf_path, engine='kaleido', scale=2.4)

    # free up memory
    del w_pca_fig

    # plot window stats & save
    from utils import plot_w_stats
    w_stats_fig = plot_w_stats(w_stats_df, chrom, start, stop, w_size, w_step)
    w_stats_fig.write_html(w_stats_fig_html_path)
    w_stats_fig.write_image(w_stats_fig_pdf_path, engine='kaleido', scale=2.4)

if __name__ == "__main__":
    main()

[INFO] Processed 1 of 3400 windows
[INFO] Processed 2 of 3400 windows
[INFO] Processed 3 of 3400 windows
[INFO] Processed 4 of 3400 windows
[INFO] Processed 5 of 3400 windows
[INFO] Processed 6 of 3400 windows
[INFO] Processed 7 of 3400 windows
[INFO] Processed 8 of 3400 windows
[INFO] Processed 9 of 3400 windows
[INFO] Processed 10 of 3400 windows
[INFO] Processed 11 of 3400 windows
[INFO] Processed 12 of 3400 windows
[INFO] Processed 13 of 3400 windows
[INFO] Processed 14 of 3400 windows
[INFO] Processed 15 of 3400 windows
[INFO] Processed 16 of 3400 windows
[INFO] Processed 17 of 3400 windows
[INFO] Processed 18 of 3400 windows
[INFO] Processed 19 of 3400 windows
[INFO] Processed 20 of 3400 windows
[INFO] Processed 21 of 3400 windows
[INFO] Processed 22 of 3400 windows
[INFO] Processed 23 of 3400 windows
[INFO] Processed 24 of 3400 windows
[INFO] Processed 25 of 3400 windows
[INFO] Processed 26 of 3400 windows
[INFO] Processed 27 of 3400 windows
[INFO] Processed 28 of 3400 windows
[

In [31]:
w_pca_df

,500000,510000,520000,530000,540000,550000,560000,570000,580000,590000,...,34400000,34410000,34420000,34430000,34440000,34450000,34460000,34470000,34480000,34490000
id,,,,,,,,,,,,,,,,,,,,,
ind_1,5.252987,-5.252987,-5.241728,-5.241728,-5.241728,-5.241728,-5.463248,-5.636689,-5.643866,-5.608573,...,-8.492423,-7.018608,-7.018608,-7.005031,-7.005031,-6.988921,-6.67802,-6.758723,-7.085608,-7.141663
ind_2,4.290106,-4.290106,-4.395182,-4.395182,-4.395182,-4.395182,-4.335404,-4.273799,-4.353685,-4.247599,...,-6.144078,-6.197875,-6.197875,-6.193985,-6.193985,-6.187108,-6.147841,-6.072806,-6.415628,-6.344414
ind_3,5.294878,-5.294878,-5.297477,-5.297477,-5.297477,-5.297477,-5.339084,-5.362118,-5.096672,-5.0735,...,-4.7775,-4.945492,-4.945492,-4.909713,-4.909713,-4.875697,-4.89895,-4.858013,-4.81229,-4.743016
ind_4,4.561086,-4.561086,-4.592175,-4.592175,-4.592175,-4.592175,-4.573877,-4.546539,-4.590673,-4.551465,...,-3.982832,-4.167541,-4.167541,-4.123552,-4.123552,-4.08288,-4.12231,-4.107224,-4.07908,-4.072603
ind_5,2.971991,-2.971991,-3.119704,-3.119704,-3.119704,-3.119704,-2.862842,-2.625752,-2.658028,-2.541769,...,-1.375704,-1.681306,-1.681306,-1.65103,-1.65103,-1.624317,-1.386471,-1.217824,-1.337254,-1.202189
ind_6,5.35006,-5.35006,-5.458559,-5.458559,-5.458559,-5.458559,-5.235903,-5.036207,-5.098846,-5.011087,...,-4.156775,-5.008036,-5.008036,-4.893957,-4.893957,-4.790757,-5.186701,-5.279856,-4.18334,-4.156359
ind_7,-1.868002,1.868002,1.796442,1.796442,1.796442,1.796442,1.622347,1.396324,1.365279,1.446953,...,9.471174,9.650515,9.650515,9.670851,9.670851,9.67915,10.511088,11.306049,11.340399,11.995109
ind_8,-9.786914,9.786914,11.020192,11.020192,11.020192,11.020192,10.575745,10.190557,10.183208,9.017421,...,12.540736,12.780464,12.780464,13.041508,13.041508,13.262346,11.375856,10.105575,9.762806,8.629258
ind_9,-16.064516,16.064516,15.286482,15.286482,15.286482,15.286482,15.610721,15.892811,15.891883,16.56822,...,6.91503,6.585533,6.585533,6.062639,6.062639,5.605974,6.530861,6.880184,6.807981,7.033758


In [32]:
w_pca_tsv_path =        output_prefix + '.w_pc_' + str(pc) + '.tsv.gz'
pd.read_csv(w_pca_tsv_path,  sep='\t', index_col=None, na_values='NA')

,id,500000,510000,520000,530000,540000,550000,560000,570000,580000,...,34400000,34410000,34420000,34430000,34440000,34450000,34460000,34470000,34480000,34490000
0,ind_1,5.252987,-5.252987,-5.241728,-5.241728,-5.241728,-5.241728,-5.463248,-5.636689,-5.643866,...,-8.492423,-7.018608,-7.018608,-7.005031,-7.005031,-6.988921,-6.678020,-6.758723,-7.085608,-7.141663
1,ind_2,4.290106,-4.290106,-4.395182,-4.395182,-4.395182,-4.395182,-4.335404,-4.273799,-4.353685,...,-6.144078,-6.197875,-6.197875,-6.193985,-6.193985,-6.187108,-6.147841,-6.072806,-6.415628,-6.344414
2,ind_3,5.294878,-5.294878,-5.297477,-5.297477,-5.297477,-5.297477,-5.339084,-5.362118,-5.096672,...,-4.777500,-4.945492,-4.945492,-4.909713,-4.909713,-4.875697,-4.898950,-4.858013,-4.812290,-4.743016
3,ind_4,4.561086,-4.561086,-4.592175,-4.592175,-4.592175,-4.592175,-4.573877,-4.546539,-4.590673,...,-3.982832,-4.167541,-4.167541,-4.123552,-4.123552,-4.082880,-4.122310,-4.107224,-4.079080,-4.072603
4,ind_5,2.971991,-2.971991,-3.119704,-3.119704,-3.119704,-3.119704,-2.862842,-2.625752,-2.658028,...,-1.375704,-1.681306,-1.681306,-1.651030,-1.651030,-1.624317,-1.386471,-1.217824,-1.337254,-1.202189
5,ind_6,5.350060,-5.350060,-5.458559,-5.458559,-5.458559,-5.458559,-5.235903,-5.036207,-5.098846,...,-4.156775,-5.008036,-5.008036,-4.893957,-4.893957,-4.790757,-5.186701,-5.279856,-4.183340,-4.156359
6,ind_7,-1.868002,1.868002,1.796442,1.796442,1.796442,1.796442,1.622347,1.396324,1.365279,...,9.471174,9.650515,9.650515,9.670851,9.670851,9.679150,10.511088,11.306049,11.340399,11.995109
7,ind_8,-9.786914,9.786914,11.020192,11.020192,11.020192,11.020192,10.575745,10.190557,10.183208,...,12.540736,12.780464,12.780464,13.041508,13.041508,13.262346,11.375856,10.105575,9.762806,8.629258
8,ind_9,-16.064516,16.064516,15.286482,15.286482,15.286482,15.286482,15.610721,15.892811,15.891883,...,6.915030,6.585533,6.585533,6.062639,6.062639,5.605974,6.530861,6.880184,6.807981,7.033758


In [33]:
w_pca_df.to_csv(w_pca_tsv_path + '_2', sep='\t', compression='gzip')

In [34]:
w_pca_df.isnull().values.any()

True

In [36]:
w_stats_df

,pct_explained_pc_1,pct_explained_pc_2,num_variants,pct_included_sites
window_mid,,,,
500000,24.752402,19.138455,118,0.000001
510000,24.752402,19.138455,118,0.000001
520000,24.752201,19.575846,119,0.000001
530000,24.752201,19.575846,119,0.000001
540000,24.752201,19.575846,119,0.000001
...,...,...,...,...
34450000,22.322288,17.913805,113,0.000001
34460000,22.104089,17.136005,109,0.000001
34470000,22.035465,16.730970,108,0.000001


In [44]:
w_stats_tsv_path =      output_prefix + '.w_stats'          + '.tsv.gz'
w_stats_df_2 = pd.read_csv(w_stats_tsv_path,  sep='\t', index_col=0, na_values='NA')
w_stats_df_2

,pct_explained_pc_1,pct_explained_pc_2,num_variants,pct_included_sites
window_mid,,,,
500000,24.752402,19.138455,118,0.000001
510000,24.752402,19.138455,118,0.000001
520000,24.752201,19.575846,119,0.000001
530000,24.752201,19.575846,119,0.000001
540000,24.752201,19.575846,119,0.000001
...,...,...,...,...
34450000,22.322288,17.913805,113,0.000001
34460000,22.104089,17.136005,109,0.000001
34470000,22.035465,16.730970,108,0.000001
